In [5]:
import pandas as pd
import numpy as np
import matplotlib
import os

# OUTPUT OF ALTERYX WORKFLOWS

In [6]:
output_csv_dir = '../output/csv'
output_pickle_dir = '../input/pickle'

for path, subdirs, files in os.walk(output_csv_dir):
    for name in files:        
        df_name = name.split('.')[0] #get df name from filename
        df = pd.read_csv(os.path.join(path, name))        
        df.to_pickle(os.path.join(output_pickle_dir, df_name))

In [7]:
Alerts = pd.read_pickle(os.path.join(output_pickle_dir, 'Alerts'))
dP_siteSummarised = pd.read_pickle(os.path.join(output_pickle_dir, 'dP_siteSummarised'))
iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))
MTP_WF003_05_Visualise = pd.read_pickle(os.path.join(output_pickle_dir, 'MTP_WF003_05_Visualise Baseline Plan_ShiftData'))
sP_portSummarisedCofill = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedCofill'))
sP_portSummarisedIH = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedIH'))
sP_portSummarisedIHLineLoading = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedIHLineLoading'))


#### ALERTS

Mapping between Buckets and Periods. that will give you the period

| Field | Desc   | Source / Calc |
|-------|--------|---------------|
|Year|||              
|CCEP_Period4/5 whole week period in the calendar ||
|Level2Region| North / South / Scotland| Mapping table in alteryx|
|Site_SAP|Production Facility||
|Line|prod line within site||
|Company|||
|Numerator|Number of units | pulling this number from iP_Summarised|
|Numerator_Desc |||
|Denominator |||
|Denominator_Desc|||
|AlertType|||
|Value|||
|Lane|||
|SNPPL01|||
|Summary_Portfolio|||






In [24]:
dP_siteSummarised.KeyField.value_counts(dropna=False)

Total        2392
Available    2392
Name: KeyField, dtype: int64

In [26]:
dP_siteSummarised.KeyField.value_counts(dropna=False)

Total        2392
Available    2392
Name: KeyField, dtype: int64

In [27]:
# filter for Uom and KeyField
df = dP_siteSummarised.loc[(dP_siteSummarised.Uom == 'ZUC') & (dP_siteSummarised.KeyField=='Available')]

   

In [28]:
pivot_cols = ['Year_Period']
pivot_rows = ['Summary_Portfolio']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC', 'KeyField':'Available'}


In [10]:
# Demand Plan

pivot_cols = ['KeyField']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = dP_siteSummarised.loc[(dP_siteSummarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

demand_plan = pd.DataFrame(plot_df.to_records())

demand_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in demand_plan.columns]

#Calcuated fields
demand_plan['Stock_Availability'] = (demand_plan['Available']*100 / demand_plan['Total']).astype(int)

demand_plan


,Summary_Portfolio,Year_Period,Available,Total,Stock_Availability
0,Aseptic,2020_12,100507.0,100507.0,100
1,Aseptic,2021_01,76587.0,76587.0,100
2,Aseptic,2021_02,63142.0,63142.0,100
3,Aseptic,2021_03,101088.0,101088.0,100
4,Aseptic,2021_04,91946.0,91946.0,100
...,...,...,...,...,...
284,Water,2021_12,821267.0,821267.0,100
285,Water,2022_01,619945.0,619945.0,100
286,Water,2022_02,709895.0,709895.0,100
287,Water,2022_03,1014121.0,1014121.0,100


In [29]:
mP_Summarised.FromCountry.value_counts(dropna=False)

GB     339820
NaN     11895
FR       3200
IE       1530
NL       1515
BE       1195
IN         90
Name: FromCountry, dtype: int64

In [33]:
mP_Summarised.loc[mP_Summarised.FromCountry.isna()].ToCountry.value_counts(dropna=False)

GB    11895
Name: ToCountry, dtype: int64

In [68]:
# # Movement Plan

# pivot_cols = ['Move_Type']
# pivot_rows = ['Summary_Portfolio', 'Year_Period']
# value_col = ['Value']
# agg_func = np.sum
# filter_by = {'Uom':'ZUC'}
# #filter_by = {'Uom':'ZUC', 'FromCountry':'', '':''}

# mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

# # Conditional categories
# conditions = [
#     (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] != 'GB'),  #Exports
#     (mP_Summarised['FromCountry'] != 'GB') & (mP_Summarised['ToCountry'] == 'GB'),  #Imports
#     (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] == 'GB') & (mP_Summarised['ToRepack_Flag'] == True),
#     (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] == 'GB') & (mP_Summarised['ToRepack_Flag'] == False)
#     ]
# choices = ['Export', 'Import', 'Domestic Repack', 'IBT']

# mP_Summarised['Move_Type'] = np.select(conditions, choices, default='Move_Other')


# df = mP_Summarised.loc[(mP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

# plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
#                     columns=pivot_cols, aggfunc=agg_func)

# movement_plan = pd.DataFrame(plot_df.to_records())

# movement_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in movement_plan.columns]

# movement_plan.fillna(0)

In [16]:
#' Function to summarise dataframe for visualisation purpose.
def summarise_df_for_visualisation(df):
    unwanted_cols = ['Material', 'Value']
    cols = [ele for ele in df.columns.tolist() if ele not in unwanted_cols] 
    
    df['Value'] = df['Value'].fillna(0)
    df[cols] = df[cols].fillna('') 
        
    df = df[cols+['Value']].groupby(cols).sum().round().reset_index()    

    return df

In [70]:
pivot_cols = ['Move_Type']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

# Read pickle file and summarise dataframe
df = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))
df = summarise_df_for_visualisation(df)
df['ToRepack_Flag'] = df['ToRepack_Flag'].astype(str)


# Conditional categories used to categorise a column into categories    
cofill_sites = ['AILI_IE','APPL_GB','CACO_FR','COTT_GB','DAYL_GB','DIS_NL','EDRI_GB','KART_GE','MBEV_IN','MILK_GB',
'NORB_GB','PRIN_GB','WILD_GE','WINT_NL',]
cofill_sites = [x.strip() for x in cofill_sites]   
cofill_sites = [x.split('_')[0] for x in cofill_sites]    

import_sites = ['15N','19B','44F']


# Specify conditions list
conditions = [
    (df['FromCountry'] == 'GB') & (df['ToCountry'] != 'GB'), #Exports
    (df['Site_SAP_From'].isin(cofill_sites) | df['Site_SAP_To'].str.contains('Cofill')), #Cofill
    (df['Site_SAP_From'].isin(import_sites) | df['Site_SAP_To'].str.contains('Import')), #Imports
    (df['FromCountry'] == 'GB') & (df['ToCountry'] == 'GB') & (df['ToRepack_Flag'] == 'True'), #Domestic Repack
    (df['FromCountry'] == 'GB') & (df['ToCountry'] == 'GB') & (df['ToRepack_Flag'] == 'False') #IBT
    ]
choices = ['Export', 'Cofill', 'Import', 'Domestic Repack', 'IBT']


df['Move_Type'] = np.select(conditions, choices, default='Move_Other')

df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

movement_plan = pd.DataFrame(plot_df.to_records())

movement_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in movement_plan.columns]

movement_plan = movement_plan.fillna(0)

movement_plan

,Summary_Portfolio,Year_Period,Cofill,Export,IBT,Import,Move_Other
0,Aseptic,2020_12,0.0,108733.0,17185.0,57718.0,0.0
1,Aseptic,2021_01,7892.0,88673.0,38355.0,64430.0,0.0
2,Aseptic,2021_02,27725.0,95561.0,64133.0,72707.0,0.0
3,Aseptic,2021_03,40515.0,118730.0,93792.0,64683.0,0.0
4,Aseptic,2021_04,31361.0,112915.0,81539.0,78750.0,0.0
...,...,...,...,...,...,...,...
332,Water,2021_12,0.0,9699.0,885324.0,0.0,0.0
333,Water,2022_01,0.0,8077.0,802007.0,0.0,0.0
334,Water,2022_02,0.0,8734.0,927809.0,0.0,0.0
335,Water,2022_03,0.0,11109.0,1202731.0,0.0,0.0


In [61]:
other = df.loc[df.Move_Type == 'Move_Other']
other

,Unnamed: 0,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Uom,ToRepack_Flag,Value,Move_Type
193406,193406,,GB,,,North,DIS_,5569,2020_12,GB,ZUC,ZUC,False,7267.0,Move_Other
193411,193411,,GB,,,North,DIS_,5569,2020_12,GB,ZUC,ZUC,False,3963.0,Move_Other
193416,193416,,GB,,,North,DIS_,5569,2020_12,GB,ZUC,ZUC,False,54633.0,Move_Other
193421,193421,,GB,,,North,DIS_,5569,2020_12,GB,ZUC,ZUC,False,17816.0,Move_Other
193426,193426,,GB,,,North,DIS_,5569,2020_12,GB,ZUC,ZUC,False,24528.0,Move_Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205276,205276,,GB,,North,South,5567,5559,2022_03,REPACK,ZUC,ZUC,False,4430.0,Move_Other
205281,205281,,GB,,North,South,5567,5559,2022_04,GB,ZUC,ZUC,False,1797.0,Move_Other
205286,205286,,GB,,North,South,5567,5559,2022_04,REPACK,ZUC,ZUC,False,2868.0,Move_Other
205291,205291,,GB,,North,South,5567,5559,2022_04,REPACK,ZUC,ZUC,False,2488.0,Move_Other


In [71]:
# Production Plan In House

pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedIH.loc[(sP_portSummarisedIH[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

prod_plan_inhouse = pd.DataFrame(plot_df.to_records())

prod_plan_inhouse.columns = [hdr.replace("('Value', '", "Prod_").replace("')", "") \
                     for hdr in prod_plan_inhouse.columns]

prod_plan_inhouse


,Summary_Portfolio,Year_Period,Prod_ZUC
0,Aseptic,2020_12,25397.0
1,Aseptic,2021_01,44835.0
2,Aseptic,2021_02,50884.0
3,Aseptic,2021_03,96715.0
4,Aseptic,2021_04,91131.0
...,...,...,...
287,Water,2021_12,802581.0
288,Water,2022_01,726491.0
289,Water,2022_02,896462.0
290,Water,2022_03,1031848.0


In [72]:
# Production Plan  Cofill

pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedCofill.loc[(sP_portSummarisedCofill[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

prod_plan_cofill = pd.DataFrame(plot_df.to_records())

prod_plan_cofill.columns = [hdr.replace("('Value', '", "Cofill_").replace("')", "") \
                     for hdr in prod_plan_cofill.columns]

prod_plan_cofill

,Summary_Portfolio,Year_Period,Cofill_ZUC
0,Aseptic,2020_12,52368.0
1,Aseptic,2021_01,28833.0
2,Aseptic,2021_02,51654.0
3,Aseptic,2021_03,73170.0
4,Aseptic,2021_04,56946.0
...,...,...,...
112,SPIRITS,2021_09,2966.0
113,SPIRITS,2021_10,2226.0
114,SPIRITS,2021_11,1748.0
115,SPIRITS,2021_12,2346.0


In [101]:
iP_Summarised.head()

,Unnamed: 0,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period,Inventory_Type
0,0,Inventory_MaterialLevel,NaN,AS&D,NRB GLASS (IH),GBP_ClosingCostOfCapital_Sum,0.00000,GBP,GB,Glass,201052,Glass,2021_09,Inventory_Other
1,1,Inventory_MaterialLevel,NaN,AS&D,NRB GLASS (IH),GBP_ClosingStockValue_Avg,0.00000,GBP,GB,Glass,201052,Glass,2021_09,Inventory_Closing
2,2,Inventory_MaterialLevel,NaN,AS&D,NRB GLASS (IH),GBP_CostOfCapital_Sum,0.00000,GBP,GB,Glass,201052,Glass,2021_09,Inventory_Other
3,3,Inventory_MaterialLevel,NaN,AS&D,NRB GLASS (IH),GBP_StockValue_Avg,0.00000,GBP,GB,Glass,201052,Glass,2021_09,Inventory_Other
4,4,Inventory_MaterialLevel,NaN,AS&D,NRB GLASS (IH),PAL_ClosingStock,0.86749,PAL,GB,Glass,201052,Glass,2021_09,Inventory_Closing


In [105]:
iP_Summarised.loc[(iP_Summarised.Summary_Portfolio == 'LPET') 
                  & (iP_Summarised.Year_Period == '2020_12')
                 & (iP_Summarised.KeyField == 'ZUC_ClosingStock')
                 ]

,Unnamed: 0,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period,Inventory_Type
82736,82736,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_ClosingStock,0.0,ZUC,GB,LPET,232676,LPET,2020_12,Inventory_Closing
82739,82739,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_ClosingStock,30.0,ZUC,GB,LPET,235744,LPET,2020_12,Inventory_Closing
82756,82756,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_ClosingStock,64.0,ZUC,GB,LPET,214385,LPET,2020_12,Inventory_Closing
82757,82757,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_ClosingStock,64.0,ZUC,GB,LPET,224404,LPET,2020_12,Inventory_Closing
82787,82787,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_ClosingStock,169.0,ZUC,GB,LPET,219079,LPET,2020_12,Inventory_Closing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042246,1042246,Inventory_MaterialLevel,South,5559,LARGE PET >=1L (IH),ZUC_ClosingStock,32181.0,ZUC,GB,LPET,215666,LPET,2020_12,Inventory_Closing
1042268,1042268,Inventory_MaterialLevel,South,5559,LARGE PET >=1L (IH),ZUC_ClosingStock,34820.0,ZUC,GB,LPET,459225,LPET,2020_12,Inventory_Closing
1042279,1042279,Inventory_MaterialLevel,South,5559,LARGE PET >=1L (IH),ZUC_ClosingStock,35564.0,ZUC,GB,LPET,217852,LPET,2020_12,Inventory_Closing
1042305,1042305,Inventory_MaterialLevel,South,5559,LARGE PET >=1L (IH),ZUC_ClosingStock,40007.0,ZUC,GB,LPET,216537,LPET,2020_12,Inventory_Closing


In [73]:
# Inventory Plan 

pivot_cols = ['Inventory_Type']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
# Conditional categories
conditions = [
    iP_Summarised.KeyField.str.contains('ClosingStock', case=False),
    iP_Summarised.KeyField.str.contains('StockVsTarget', case=False)
    ]
choices = ['Inventory_Closing','Inventory_National_Target']

iP_Summarised['Inventory_Type'] = np.select(conditions, choices, default='Inventory_Other')

df = iP_Summarised.loc[(iP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

inventory_plan = pd.DataFrame(plot_df.to_records())

inventory_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_plan.columns]

inventory_plan

,Summary_Portfolio,Year_Period,Inventory_Closing,Inventory_National_Target,Inventory_Other
0,Aseptic,2020_12,178228.0,20086.0,126889.414286
1,Aseptic,2021_01,167362.0,1966.0,90000.200000
2,Aseptic,2021_02,202296.0,120.0,95410.500000
3,Aseptic,2021_03,231864.0,52.0,111415.600000
4,Aseptic,2021_04,260162.0,15686.0,124625.750000
...,...,...,...,...,...
289,Water,2021_12,1150336.0,0.0,718162.633333
290,Water,2022_01,1311730.0,0.0,650910.750000
291,Water,2022_02,1565850.0,60628.0,762537.500000
292,Water,2022_03,1586207.0,-1.0,977445.800000


In [100]:
x = inventory_plan.loc[inventory_plan.Summary_Portfolio == 'LPET']

,Summary_Portfolio,Year_Period,Inventory_Closing,Inventory_National_Target,Inventory_Other
119,LPET,2020_12,4937316.00,0.00,3.277860e+06
120,LPET,2021_01,6510050.00,0.00,2.779017e+06
121,LPET,2021_02,7868538.00,137836.00,3.623108e+06
122,LPET,2021_03,7793431.00,-1.00,3.929183e+06
123,LPET,2021_04,7119697.18,20213.18,3.749781e+06
124,LPET,2021_05,7401421.18,1.18,3.672031e+06
125,LPET,2021_06,7648394.18,0.18,3.738413e+06
126,LPET,2021_07,7278685.66,-0.34,3.708831e+06
127,LPET,2021_08,6642705.66,2797.66,3.448698e+06
128,LPET,2021_09,6500466.66,-1.34,3.314797e+06


In [17]:
# Days of Stock Inventory Plan - Projected stock holding 

pivot_cols = ['DSI_Type']
pivot_rows = ['Line_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {}

iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
# Conditional categories
conditions = [
    (iP_Summarised.KeyField == 'Days_DSI_National_LinePortfolio'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'North'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'South'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'Scotland'),
    (iP_Summarised.KeyField == 'Days_ClosingStockTarget_National_LinePortfolio')
    ]
choices = ['DSI_National','DSI_North', 'DSI_South', 'DSI_Scotland', 'DSI_Target_National']

iP_Summarised['DSI_Type'] = np.select(conditions, choices, default='DSI_Other')

df = iP_Summarised.loc[(iP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

dsi_plan = pd.DataFrame(plot_df.to_records())

dsi_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in dsi_plan.columns]

dsi_plan

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Line_Portfolio,Year_Period,DSI_Other
0,Aseptic,2020_12,1.582762e+06
1,Aseptic,2021_01,1.206132e+06
2,Aseptic,2021_02,1.341454e+06
3,Aseptic,2021_03,1.549792e+06
4,Aseptic,2021_04,1.788937e+06
...,...,...,...
221,Water,2021_12,4.519310e+06
222,Water,2022_01,4.671008e+06
223,Water,2022_02,5.694370e+06
224,Water,2022_03,6.197908e+06


In [84]:
combined_df.Cofill_ZUC.isna().value_counts()

True     200
False     89
Name: Cofill_ZUC, dtype: int64

In [86]:
print(demand_plan.shape)
print(movement_plan.shape)
print(prod_plan_inhouse.shape)
print(prod_plan_cofill.shape)
print(inventory_plan.shape)

(289, 5)
(337, 7)
(292, 3)
(117, 3)
(294, 5)


In [94]:
demand_plan.Summary_Portfolio.value_counts()

Aseptic           17
REPACK            17
OTHER             17
GB                17
Cans - Singles    17
Glass             17
SPET              17
Monster           17
Cans - Shrink     17
Water             17
LPET              17
Relentless        17
Capri Sun         17
Cans - Board      17
Postmix           17
Oasis             17
Small Cans        17
Name: Summary_Portfolio, dtype: int64

In [93]:
movement_plan.Summary_Portfolio.value_counts()

GB                17
Aseptic           17
SPET              17
Monster           17
Cans - Shrink     17
Small Cans        17
Capri Sun         17
Cans - Board      17
Postmix           17
Water             17
REPACK            17
SE                17
OTHER             17
Cans - Singles    17
Glass             17
LPET              17
Relentless        17
Oasis             17
SPIRITS           14
BE                14
NL                 2
IS                 1
Name: Summary_Portfolio, dtype: int64

In [95]:
# Merge DFs into one

dfs = [demand_plan, movement_plan, prod_plan_inhouse, prod_plan_cofill, inventory_plan]
dfs = [df.set_index(['Summary_Portfolio', 'Year_Period']) for df in dfs]
combined_df = dfs[0].join(dfs[1:]).reset_index()
combined_df = combined_df.fillna(0)

#combined_df = combined_df.loc[combined_df.Summary_Portfolio == 'LPET']
# combined_df = combined_df[['Summary_Portfolio','Year_Period','Available',
#                            'Export','Import','Prod_ZUC', 
#                            'Cofill_ZUC','Inventory_Closing']]
combined_df

,Summary_Portfolio,Year_Period,Available,Total,Stock_Availability,Cofill,Export,IBT,Import,Move_Other,Prod_ZUC,Cofill_ZUC,Inventory_Closing,Inventory_National_Target,Inventory_Other
0,Aseptic,2020_12,100507.0,100507.0,100.0,0.0,108733.0,17185.0,57718.0,0.0,25397.0,52368.0,178228.0,20086.0,126889.414286
1,Aseptic,2021_01,76587.0,76587.0,100.0,7892.0,88673.0,38355.0,64430.0,0.0,44835.0,28833.0,167362.0,1966.0,90000.200000
2,Aseptic,2021_02,63142.0,63142.0,100.0,27725.0,95561.0,64133.0,72707.0,0.0,50884.0,51654.0,202296.0,120.0,95410.500000
3,Aseptic,2021_03,101088.0,101088.0,100.0,40515.0,118730.0,93792.0,64683.0,0.0,96715.0,73170.0,231864.0,52.0,111415.600000
4,Aseptic,2021_04,91946.0,91946.0,100.0,31361.0,112915.0,81539.0,78750.0,0.0,91131.0,56946.0,260162.0,15686.0,124625.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Water,2021_12,821267.0,821267.0,100.0,0.0,9699.0,885324.0,0.0,0.0,802581.0,0.0,1150336.0,0.0,718162.633333
285,Water,2022_01,619945.0,619945.0,100.0,0.0,8077.0,802007.0,0.0,0.0,726491.0,0.0,1311730.0,0.0,650910.750000
286,Water,2022_02,709895.0,709895.0,100.0,0.0,8734.0,927809.0,0.0,0.0,896462.0,0.0,1565850.0,60628.0,762537.500000
287,Water,2022_03,1014121.0,1014121.0,100.0,0.0,11109.0,1202731.0,0.0,0.0,1031848.0,0.0,1586207.0,-1.0,977445.800000


In [96]:
combined_df['Cofill'] = combined_df['Cofill'] + combined_df['Cofill_ZUC']
combined_df = combined_df.drop(['Cofill_ZUC'], axis=1)
combined_df

,Summary_Portfolio,Year_Period,Available,Total,Stock_Availability,Cofill,Export,IBT,Import,Move_Other,Prod_ZUC,Inventory_Closing,Inventory_National_Target,Inventory_Other
0,Aseptic,2020_12,100507.0,100507.0,100.0,52368.0,108733.0,17185.0,57718.0,0.0,25397.0,178228.0,20086.0,126889.414286
1,Aseptic,2021_01,76587.0,76587.0,100.0,36725.0,88673.0,38355.0,64430.0,0.0,44835.0,167362.0,1966.0,90000.200000
2,Aseptic,2021_02,63142.0,63142.0,100.0,79379.0,95561.0,64133.0,72707.0,0.0,50884.0,202296.0,120.0,95410.500000
3,Aseptic,2021_03,101088.0,101088.0,100.0,113685.0,118730.0,93792.0,64683.0,0.0,96715.0,231864.0,52.0,111415.600000
4,Aseptic,2021_04,91946.0,91946.0,100.0,88307.0,112915.0,81539.0,78750.0,0.0,91131.0,260162.0,15686.0,124625.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Water,2021_12,821267.0,821267.0,100.0,0.0,9699.0,885324.0,0.0,0.0,802581.0,1150336.0,0.0,718162.633333
285,Water,2022_01,619945.0,619945.0,100.0,0.0,8077.0,802007.0,0.0,0.0,726491.0,1311730.0,0.0,650910.750000
286,Water,2022_02,709895.0,709895.0,100.0,0.0,8734.0,927809.0,0.0,0.0,896462.0,1565850.0,60628.0,762537.500000
287,Water,2022_03,1014121.0,1014121.0,100.0,0.0,11109.0,1202731.0,0.0,0.0,1031848.0,1586207.0,-1.0,977445.800000


In [98]:
combined_df['DEMAND'] = combined_df['Total'] + combined_df['Export']  #+ combined_df['Domestic Repack']
combined_df['SUPPLY'] = combined_df['Prod_ZUC'] + combined_df['Import']  + combined_df['Cofill']
combined_df['DIFF'] = combined_df['SUPPLY'] - combined_df['DEMAND'] 

In [106]:
combined_df[['Summary_Portfolio','Year_Period','DEMAND', 'SUPPLY', 'DIFF', 'Inventory_Closing']]

,Summary_Portfolio,Year_Period,DEMAND,SUPPLY,DIFF,Inventory_Closing
0,Aseptic,2020_12,209240.0,135483.0,-73757.0,178228.0
1,Aseptic,2021_01,165260.0,145990.0,-19270.0,167362.0
2,Aseptic,2021_02,158703.0,202970.0,44267.0,202296.0
3,Aseptic,2021_03,219818.0,275083.0,55265.0,231864.0
4,Aseptic,2021_04,204861.0,258188.0,53327.0,260162.0
...,...,...,...,...,...,...
284,Water,2021_12,830966.0,802581.0,-28385.0,1150336.0
285,Water,2022_01,628022.0,726491.0,98469.0,1311730.0
286,Water,2022_02,718629.0,896462.0,177833.0,1565850.0
287,Water,2022_03,1025230.0,1031848.0,6618.0,1586207.0


In [75]:
key_cols = ['Summary_Portfolio', 'Year_Period']
value_cols = list(set(combined_df.columns) - set(key_cols))

value_cols

['Export',
 'Move_Other',
 'Prod_ZUC',
 'Inventory_National_Target',
 'Total',
 'Available',
 'Cofill',
 'Import',
 'Inventory_Closing',
 'Inventory_Other',
 'IBT',
 'Stock_Availability',
 'Cofill_ZUC']

In [76]:
long_df = combined_df.melt(id_vars=key_cols, 
              value_vars=value_cols,
              var_name='Category', value_name='Value')

In [77]:
long_df['Parent_Category'] = ['Demand' if x in ['Total','Domestic Repack', 'Export'] \
                                       else 'Supply' if x in ['Prod_ZUC','Import', 'Cofill_ZUC', 'Cofill'] \
                                       else '' for x in long_df.Category]

In [78]:
demand_supply_df = long_df.loc[long_df.Parent_Category.isin(['Demand', 'Supply'])]
inventory_df =  combined_df[['Summary_Portfolio','Year_Period', 'Inventory_Closing','Inventory_National_Target','Inventory_Other']]

plot_df = pd.merge(demand_supply_df, inventory_df, on=['Summary_Portfolio','Year_Period'], how='inner')

plot_df = plot_df[plot_df.Summary_Portfolio == 'Aseptic']

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

#fig = fig = make_subplots(specs=[[{"secondary_y": True}]]) #go.Figure()
fig = go.Figure()

fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Year Period"),
    yaxis=dict(title_text="Units"),
    barmode="stack",
)

colors = sns.color_palette("rocket").as_hex() #["#2A66DE","#fd5f42", "#c55f42", "#c58a42", "#c5c142", "#FFC32B"]


for r, c in zip(plot_df.Category.unique(), colors):   
    this_df = plot_df[plot_df.Category == r]
    
    fig.add_trace(
        go.Bar(x=[this_df.Year_Period, this_df.Parent_Category], y=this_df.Value, name=r, marker_color=c),
    )

fig.add_trace(
        go.Scatter(x=[plot_df.Year_Period, plot_df.Parent_Category] ,y=plot_df.Inventory_Closing, name='Inventory_Closing')
)

    
fig

In [170]:
dsi_plan

,Line_Portfolio,Year_Period,DSI_National,DSI_North,DSI_Other,DSI_Scotland,DSI_South,DSI_Target_National
0,Aseptic,2020_06,33.185422,33.633408,2.209621e+06,35.291952,32.583292,35.023473
1,Aseptic,2020_07,30.885199,31.375978,2.163220e+06,35.023031,30.143762,34.251959
2,Aseptic,2020_08,31.863403,32.293428,2.133879e+06,35.000000,31.220210,34.457008
3,Aseptic,2020_09,50.422734,35.968882,2.842262e+06,34.690327,67.089994,33.992982
4,Aseptic,2020_10,37.259292,34.822545,2.186059e+06,34.231907,40.249502,33.893940
...,...,...,...,...,...,...,...,...
185,Water,2021_04,28.093982,28.077135,1.206996e+06,28.146630,28.112287,28.000000
186,Water,2021_05,0.000000,0.000000,4.408414e+03,0.000000,0.000000,0.000000
187,Water,2021_06,0.000000,0.000000,4.409987e+03,0.000000,0.000000,0.000000
188,Water,2021_07,0.000000,0.000000,4.408414e+03,0.000000,0.000000,0.000000


In [179]:
plot_df

,Year_Period,DSI_National,DSI_North,DSI_Scotland,DSI_South,DSI_Target_National
0,2020_06,33.185422,33.633408,35.291952,32.583292,35.023473
1,2020_07,30.885199,31.375978,35.023031,30.143762,34.251959
2,2020_08,31.863403,32.293428,35.000000,31.220210,34.457008
3,2020_09,50.422734,35.968882,34.690327,67.089994,33.992982
4,2020_10,37.259292,34.822545,34.231907,40.249502,33.893940
5,2020_11,37.280134,37.974301,36.712201,36.523340,36.121583
6,2020_12,30.882899,32.022619,33.613462,29.478141,28.699245
7,2021_01,36.548685,37.987996,35.962617,34.951198,33.973441
8,2021_02,36.016706,37.386311,35.000000,34.551936,33.606270
9,2021_03,36.087641,37.395349,35.259609,34.757830,33.806623


In [188]:
plot_df = dsi_plan[dsi_plan.Line_Portfolio == 'Aseptic']
index_cols = ['Year_Period']
bar_cols = ['DSI_National','DSI_North','DSI_Scotland', 'DSI_South']
line_cols = ['DSI_Target_National']
plot_df = plot_df[index_cols + bar_cols + line_cols]

fig = go.Figure()

fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Year Period"),
    yaxis=dict(title_text="Days")    
)

colors = sns.color_palette("mako").as_hex() #["#2A66DE","#fd5f42", "#c55f42", "#c58a42", "#c5c142", "#FFC32B"]


for column, color in zip(bar_cols, colors):       
    fig.add_trace(
        go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

colors = sns.color_palette("rocket").as_hex()
for column, color in zip(line_cols, colors):       
    fig.add_trace(
        go.Scatter(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

    
fig

In [177]:
plot_df

,DSI_National,DSI_North,DSI_Scotland,DSI_South,DSI_Target_National
0,33.185422,33.633408,35.291952,32.583292,35.023473
1,30.885199,31.375978,35.023031,30.143762,34.251959
2,31.863403,32.293428,35.000000,31.220210,34.457008
3,50.422734,35.968882,34.690327,67.089994,33.992982
4,37.259292,34.822545,34.231907,40.249502,33.893940
5,37.280134,37.974301,36.712201,36.523340,36.121583
6,30.882899,32.022619,33.613462,29.478141,28.699245
7,36.548685,37.987996,35.962617,34.951198,33.973441
8,36.016706,37.386311,35.000000,34.551936,33.606270
9,36.087641,37.395349,35.259609,34.757830,33.806623


In [172]:
x = ['1','2','3', '4']
y = ['5', '6',]

for r,c in zip(y,x):
    print(r + c)

51
62


In [1]:
import pyodbc 
import pandas 

server = 'tcp:myserver.database.windows.net' 
database = 'mydb' 
username = 'myusername' 
password = 'mypassword' 
#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()


conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=CVAPWPEQR001;'
                      'Database=PACSQL_MTP;'
                      'Trusted_Connection=yes;')

sql = 'SELECT TOP 100 * FROM [PACSQL_MTP].[dbo].[temp_dP_siteSummarised_materiallevel]'

data = pandas.read_sql(sql,conn)

data.head()

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom
0,North,5502,2020_06,Aseptic,None,Available,7.901533e+04,ZRW
1,North,5502,2020_06,Aseptic,None,Available,8.489500e+04,ZUC
2,North,5502,2020_06,Aseptic,None,Total,7.901533e+04,ZRW
3,North,5502,2020_06,Aseptic,None,Total,8.489500e+04,ZUC
4,North,5502,2020_06,Cans - Board,None,Available,2.654606e+06,ZRW


In [2]:
data['Material'].value_counts(dropna=False)

NaN    100
Name: Material, dtype: int64

In [20]:
unwanted_cols = ['Material', 'Value']
cols = [ele for ele in data.columns.tolist() if ele not in unwanted_cols] 
cols

['Level2Region',
 'Site_SAP',
 'Year_Period',
 'Summary_Portfolio',
 'KeyField',
 'Uom']

In [23]:
data[cols+['Value']].groupby(cols).sum().round().reset_index()

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Uom,Value
0,North,5502,2020_06,Aseptic,Available,ZRW,79015.0
1,North,5502,2020_06,Aseptic,Available,ZUC,84895.0
2,North,5502,2020_06,Aseptic,Total,ZRW,79015.0
3,North,5502,2020_06,Aseptic,Total,ZUC,84895.0
4,North,5502,2020_06,Cans - Board,Available,ZRW,2654606.0
...,...,...,...,...,...,...,...
95,North,5502,2020_07,Glass,Total,ZUC,348517.0
96,North,5502,2020_07,LPET,Available,ZRW,2867171.0
97,North,5502,2020_07,LPET,Available,ZUC,5519767.0
98,North,5502,2020_07,LPET,Total,ZRW,2867171.0


In [11]:
old_ip = pd.read_pickle('../output/pickle/sP_portSummarisedIHLineLoading')
new_ip = pd.read_pickle('../input/pickle/sP_portSummarisedIHLineLoading')

In [24]:
old_ip.KeyField.unique()

array(['Planned', 'Capacity', 'Consumed', 'EstimatedHoursSpareCapacity',
       'EstimatedUnconsumedCapacity', 'SpareHours'], dtype=object)

In [25]:
new_ip.KeyField.unique()

array(['Planned', 'Capacity', 'Consumed', 'EstimatedHoursSpareCapacity',
       'EstimatedUnconsumedCapacity', 'SpareHours'], dtype=object)

In [2]:
dp = pd.read_pickle('../input/pickle/dP_siteSummarised')
mp = pd.read_pickle('../input/pickle/mP_Summarised')

In [4]:
dp.drop('Material_x', axis=1)

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom,Material_y
0,North,5502,2020_06,Aseptic,Available,79015.325627,ZRW,461020
1,North,5502,2020_06,Aseptic,Available,79015.325627,ZRW,413253
2,North,5502,2020_06,Aseptic,Available,79015.325627,ZRW,413273
3,North,5502,2020_06,Aseptic,Available,79015.325627,ZRW,413299
4,North,5502,2020_06,Aseptic,Available,79015.325627,ZRW,203222
...,...,...,...,...,...,...,...,...
209259,South,5560,2021_05,Cans - Board,Total,158.000000,ZUC,265750
209260,South,5560,2021_05,Cans - Board,Total,158.000000,ZUC,270518
209261,South,5560,2021_05,Cans - Board,Total,158.000000,ZUC,270519
209262,South,5560,2021_05,Cans - Board,Total,158.000000,ZUC,264092


In [14]:
not any(dp.Material != '')

True

In [30]:
def summarise_df_for_visualisation(df):
    unwanted_cols = ['Material', 'Value']
    cols = [ele for ele in df.columns.tolist() if ele not in unwanted_cols] 

    df = df[cols+['Value']].groupby(cols).sum().round().reset_index()

    return df

In [31]:
import pandas as pd
import numpy as np
import os

In [28]:
pivot_cols = ['Hours_Type']
pivot_rows = ['Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {}

sP_shiftData = pd.read_pickle('../input/pickle/MTP_WF003_05_Visualise Baseline Plan_ShiftData')

# Conditional categories
conditions = [
    sP_shiftData.KeyField == 'Hours_Spare',
    sP_shiftData.KeyField == 'Hours_DownTime', 
    sP_shiftData.KeyField == 'Hours_SchedPDT',
    sP_shiftData.KeyField == 'Hours_UnCrewed',
    ]
choices = ['Spare_Hours','Planned_Downtime', 'Loss_Factor', 'Uncrewed']

sP_shiftData['Hours_Type'] = np.select(conditions, choices, default='Hours_Other')

#'Filter'
df = sP_shiftData.loc[(sP_shiftData[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

# pivot
wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)


shift_plan = pd.DataFrame(wide_df.to_records())

shift_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in shift_plan.columns]

shift_plan = shift_plan.fillna(0)

shift_plan


C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Site_SAP,Line,Year_Period,Hours_Other,Loss_Factor,Planned_Downtime,Spare_Hours,Uncrewed
0,5501,LN01,2020_06,1415.2175,197.25,51.0,17.7525,0.0
1,5501,LN01,2020_07,1137.3650,159.50,34.0,14.3550,0.0
2,5501,LN01,2020_08,1137.3650,159.50,34.0,14.3550,0.0
3,5501,LN01,2020_09,1188.2375,119.25,363.0,10.7325,0.0
4,5501,LN01,2020_10,1137.3650,159.50,34.0,14.3550,0.0
...,...,...,...,...,...,...,...,...
475,5546,LN01,2021_04,1.1800,0.00,0.0,0.0000,672.0
476,5546,LN01,2021_05,1.1800,0.00,0.0,0.0000,672.0
477,5546,LN01,2021_06,1.1800,0.00,0.0,0.0000,840.0
478,5546,LN01,2021_07,1.1800,0.00,0.0,0.0000,672.0


In [32]:
pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'Hours'}

df = summarise_df_for_visualisation(pd.read_pickle('../input/pickle/sP_portSummarisedIHLineLoading'))
#df = sP_portSummarisedIHLineLoading.loc[(sP_portSummarisedIHLineLoading[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)




production_line_plan = pd.DataFrame(wide_df.to_records())

production_line_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in production_line_plan.columns]

#Calcuated fields
production_line_plan['Capacity_Utilisation'] = production_line_plan['Consumed']*100 / production_line_plan['Capacity']
production_line_plan['Available_Vol'] = production_line_plan['EstimatedUnconsumedCapacity']
production_line_plan['Available_Vol_with_Safety_Hours'] = production_line_plan['EstimatedUnconsumedCapacity'] + production_line_plan['EstimatedHoursSpareCapacity']


production_line_plan = production_line_plan.fillna(0)

production_line_plan

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,EstimatedHoursSpareCapacity,EstimatedUnconsumedCapacity,Planned,SpareHours,Capacity_Utilisation,Available_Vol,Available_Vol_with_Safety_Hours
0,Aseptic,5501,LN03,2020_06,504.0,504.0,33913.0,2.0,3145435.0,16.0,100.000000,2.0,33915.0
1,Aseptic,5501,LN03,2020_07,408.0,408.0,27528.0,2.0,3302802.0,13.0,100.000000,2.0,27530.0
2,Aseptic,5501,LN03,2020_08,408.0,408.0,29585.0,4.0,2233182.0,13.0,100.000000,4.0,29589.0
3,Aseptic,5501,LN03,2020_09,505.0,296.0,49074.0,656838.0,929864.0,16.0,58.613861,656838.0,705912.0
4,Aseptic,5501,LN03,2020_10,397.0,149.0,36114.0,731665.0,289068.0,12.0,37.531486,731665.0,767779.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,Water,5546,LN01,2021_02,335.0,202.0,0.0,801050.0,1209720.0,0.0,60.298507,801050.0,801050.0
427,Water,5546,LN01,2021_03,839.0,279.0,0.0,3382784.0,1677077.0,0.0,33.253874,3382784.0,3382784.0
428,Water,5546,LN01,2021_04,672.0,54.0,0.0,3641015.0,311411.0,0.0,8.035714,3641015.0,3641015.0
429,Water,5546,LN01,2021_05,672.0,12.0,0.0,3641132.0,59873.0,0.0,1.785714,3641132.0,3641132.0


In [19]:
pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = summarise_df_for_visualisation(pd.read_pickle('../input/pickle/sP_portSummarisedIHLineLoading'))
#df = sP_portSummarisedIHLineLoading.loc[(sP_portSummarisedIHLineLoading[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)


Available_Vol = pd.DataFrame(wide_df.to_records())

Available_Vol.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in Available_Vol.columns]


Available_Vol['Available_Vol'] = Available_Vol['EstimatedUnconsumedCapacity']
Available_Vol['Available_Vol_with_Safety_Hours'] = Available_Vol['EstimatedUnconsumedCapacity'] + Available_Vol['EstimatedHoursSpareCapacity']

Available_Vol = Available_Vol.fillna(0)

Available_Vol

# production_hours = pd.merge(production_line_plan, shift_plan, on=['Site_SAP', 'Line', 'Year_Period'], how='inner')

# production_hours = pd.merge(production_hours, Available_Vol, \
#                             on=['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period'], \
#                             how='inner')

# production_hours

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,EstimatedHoursSpareCapacity,EstimatedUnconsumedCapacity,Planned,SpareHours,Available_Vol,Available_Vol_with_Safety_Hours
0,Aseptic,5501,LN03,2020_06,504.0,504.0,33913.0,2.0,3145435.0,16.0,2.0,33915.0
1,Aseptic,5501,LN03,2020_07,408.0,408.0,27528.0,2.0,3302802.0,13.0,2.0,27530.0
2,Aseptic,5501,LN03,2020_08,408.0,408.0,29585.0,4.0,2233182.0,13.0,4.0,29589.0
3,Aseptic,5501,LN03,2020_09,505.0,296.0,49074.0,656838.0,929864.0,16.0,656838.0,705912.0
4,Aseptic,5501,LN03,2020_10,397.0,149.0,36114.0,731665.0,289068.0,12.0,731665.0,767779.0
...,...,...,...,...,...,...,...,...,...,...,...,...
426,Water,5546,LN01,2021_02,335.0,202.0,0.0,801050.0,1209720.0,0.0,801050.0,801050.0
427,Water,5546,LN01,2021_03,839.0,279.0,0.0,3382784.0,1677077.0,0.0,3382784.0,3382784.0
428,Water,5546,LN01,2021_04,672.0,54.0,0.0,3641015.0,311411.0,0.0,3641015.0,3641015.0
429,Water,5546,LN01,2021_05,672.0,12.0,0.0,3641132.0,59873.0,0.0,3641132.0,3641132.0


In [16]:
pivot_cols = ['KeyField']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = summarise_df_for_visualisation(pd.read_pickle('../input/pickle/dP_siteSummarised'))

df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

demand_plan = pd.DataFrame(plot_df.to_records())

demand_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in demand_plan.columns]

demand_plan

,Summary_Portfolio,Year_Period,Available,Total
0,Aseptic,2020_06,181031.0,181031.0
1,Aseptic,2020_07,187579.0,187579.0
2,Aseptic,2020_08,197000.0,197000.0
3,Aseptic,2020_09,233530.0,233530.0
4,Aseptic,2020_10,167865.0,167865.0
...,...,...,...,...
195,Water,2021_01,579360.0,579360.0
196,Water,2021_02,666389.0,666389.0
197,Water,2021_03,1002735.0,1002735.0
198,Water,2021_04,900258.0,900258.0


In [3]:
pivot_cols = ['KeyField']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

#df = summarise_df_for_visualisation(pd.read_pickle('../input/pickle/dP_siteSummarised'))

dP_siteSummarised = pd.read_pickle('../input/pickle/dP_siteSummarised')
#dP_siteSummarised
dP_siteSummarised.drop('Material', axis=1)

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom
0,North,5502,2020_06,Aseptic,Available,7.901533e+04,ZRW
1,North,5502,2020_06,Aseptic,Available,8.489500e+04,ZUC
2,North,5502,2020_06,Aseptic,Total,7.901533e+04,ZRW
3,North,5502,2020_06,Aseptic,Total,8.489500e+04,ZUC
4,North,5502,2020_06,Cans - Board,Available,2.654606e+06,ZRW
...,...,...,...,...,...,...,...
4779,South,5560,2021_04,Cans - Board,Total,6.330000e+02,ZUC
4780,South,5560,2021_05,Cans - Board,Available,1.246031e+02,ZRW
4781,South,5560,2021_05,Cans - Board,Available,1.580000e+02,ZUC
4782,South,5560,2021_05,Cans - Board,Total,1.246031e+02,ZRW


In [7]:
pickle_dir = '../input/pickle'

sP_portSummarisedIHLineLoading = pd.read_pickle(os.path.join(pickle_dir, 'sP_portSummarisedIHLineLoading'))
sP_portSummarisedIHLineLoading

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,090865,5501,LN03,2020_06,Planned,0.0,ZRW
1,Aseptic,090865,5501,LN03,2020_06,Planned,24894.0,ZUC
2,Aseptic,090865,5501,LN03,2020_07,Planned,0.0,ZRW
3,Aseptic,090865,5501,LN03,2020_07,Planned,9842.0,ZUC
4,Aseptic,090865,5501,LN03,2020_08,Planned,0.0,ZRW
...,...,...,...,...,...,...,...,...
20783,Water,,5546,LN01,2021_02,SpareHours,0.0,Hours
20784,Water,,5546,LN01,2021_03,SpareHours,0.0,Hours
20785,Water,,5546,LN01,2021_04,SpareHours,0.0,Hours
20786,Water,,5546,LN01,2021_05,SpareHours,0.0,Hours


In [8]:
iP_Summarised = pd.read_pickle(os.path.join(pickle_dir, 'iP_Summarised'))
iP_Summarised

,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period
0,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_ClosingCostOfCapital_Sum,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
1,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_ClosingStockValue_Avg,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
2,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_CostOfCapital_Sum,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
3,Inventory_MaterialLevel,,AS&D,REPACK (RP),GBP_StockValue_Avg,0.0,GBP,GB,REPACK,435648,REPACK,2020_10
4,Inventory_MaterialLevel,,AS&D,REPACK (RP),PAL_ClosingStock,0.0,PAL,GB,REPACK,435648,REPACK,2020_10
...,...,...,...,...,...,...,...,...,...,...,...,...
436251,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_StockVsTarget,0.0,ZUC,GB,REPACK,230116,REPACK,2020_12
436252,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_StockVsTarget,24781.0,ZUC,GB,REPACK,230115,REPACK,2020_11
436253,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_Stock_Avg,17221.0,ZUC,GB,REPACK,230116,REPACK,2020_09
436254,Inventory_MaterialLevel,South,5560,REPACK (RP),ZUC_Stock_Avg,20781.0,ZUC,GB,REPACK,230116,REPACK,2020_12


In [37]:
dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)
dP_siteSummarised

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom,Material
0,North,5502,2020_06,Aseptic,Available,79015.33,ZRW,461020
67,North,5502,2020_06,Aseptic,Available,84895.00,ZUC,461020
134,North,5502,2020_06,Aseptic,Total,79015.33,ZRW,461020
201,North,5502,2020_06,Aseptic,Total,84895.00,ZUC,461020
268,North,5502,2020_06,Cans - Board,Available,2654606.00,ZRW,256816
...,...,...,...,...,...,...,...,...
209079,South,5560,2021_04,Cans - Board,Total,633.00,ZUC,256816
209116,South,5560,2021_05,Cans - Board,Available,124.60,ZRW,256816
209153,South,5560,2021_05,Cans - Board,Available,158.00,ZUC,256816
209190,South,5560,2021_05,Cans - Board,Total,124.60,ZRW,256816


In [2]:
config.INPUT_PICKLE_DIR = '../input/pickle'

NameError: name 'config' is not defined

In [7]:
DEMAND_SHORTFALL_TRESHOLD = 0
INVENTORY_SHORTFALL_THRESHOLD = 5
INPUT_PICKLE_DIR = '../input/pickle'


dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)

# Demand Shortfalls
demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio', 'Material', 'Uom'], \
                                      columns='KeyField')['Value'].reset_index()

demand_plan['demand_short_fall'] = -1*(demand_plan['Available'] - demand_plan['Total'])
demand_plan['demand_short_fall_pct'] = round((demand_plan['Available'] - demand_plan['Total'])*-100/ demand_plan['Available'])

demand_shortfalls = demand_plan.loc[demand_plan.demand_short_fall > DEMAND_SHORTFALL_TRESHOLD] 

demand_shortfalls = demand_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'demand_short_fall']]
demand_shortfalls.columns = ['Demand_Site', 'Portfolio', 'Material', 'Period', 'Additional_Units_Required']


# Inventory Shortfalls
pivot_cols = ['KeyField']
pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_ClosingStockTarget', 'ZUC_StockVsTarget'])]

inventory_shortfalls = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_shortfalls = pd.DataFrame(inventory_shortfalls.to_records())

inventory_shortfalls.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_shortfalls.columns]

inventory_shortfalls['inventory_shortfall_pct'] = round(inventory_shortfalls['ZUC_StockVsTarget']*-100 / inventory_shortfalls['ZUC_ClosingStockTarget'], 2)

inventory_shortfalls['inventory_shortfall'] = -1*inventory_shortfalls['ZUC_StockVsTarget']

inventory_shortfalls = inventory_shortfalls.loc[inventory_shortfalls.inventory_shortfall_pct > INVENTORY_SHORTFALL_THRESHOLD]

inventory_shortfalls = inventory_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'inventory_shortfall']]

inventory_shortfalls.columns = ['Inventory_Site', 'Portfolio', 'Material', 'Period', 'Inventory_Units_Required']

In [31]:
line_loading.to_csv('../input/levers/shift_lever_material.csv')

In [10]:
line_loading.Line_Portfolio.unique()

array(['Aseptic', 'Cans', 'Capri Sun', 'GB', 'Glass', 'LPET', 'NL',
       'Oasis', 'Postmix', 'REPACK', 'SPET', 'Small Cans', 'Water', 'BIB'],
      dtype=object)

In [8]:
demand_shortfalls

,Demand_Site,Portfolio,Material,Period,Additional_Units_Required
1,5502,Cans - Board,106513,2020_06,1.40
6,5502,Glass,106513,2020_06,2.00
12,5502,REPACK,106513,2020_06,15197.00
32,5502,Small Cans,106513,2020_07,0.28
35,5502,Cans - Board,106513,2020_08,0.60
...,...,...,...,...,...
690,5506,Cans - Shrink,201050,2020_12,1.00
691,5506,Cans - Singles,201050,2020_12,0.67
694,5506,LPET,201050,2020_12,1.00
707,5506,Cans - Singles,201051,2021_01,0.67


In [46]:
dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)
sP_portSummarisedIHLineLoading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading')).round(2)

x = sP_portSummarisedIHLineLoading[['Site_SAP', 'Material', 'Year_Period']].drop_duplicates()    
dP_siteSummarised = dP_siteSummarised.drop('Material', axis=1)
dP_siteSummarised = pd.merge(dP_siteSummarised, x, how='inner', on=['Site_SAP', 'Year_Period'])
dP_siteSummarised = dP_siteSummarised.drop_duplicates(subset=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio',
                                      'KeyField', 'Value', 'Uom'])

dP_siteSummarised

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,KeyField,Value,Uom,Material
0,North,5502,2020_06,Aseptic,Available,79015.33,ZRW,106513
270,North,5502,2020_06,Aseptic,Available,84895.00,ZUC,106513
540,North,5502,2020_06,Aseptic,Total,79015.33,ZRW,106513
810,North,5502,2020_06,Aseptic,Total,84895.00,ZUC,106513
1080,North,5502,2020_06,Cans - Board,Available,2654606.00,ZRW,106513
...,...,...,...,...,...,...,...,...
564634,South,5506,2021_05,Small Cans,Total,29158.00,ZUC,408762
564636,South,5506,2021_05,Water,Available,22293.08,ZRW,408762
564638,South,5506,2021_05,Water,Available,51057.00,ZUC,408762
564640,South,5506,2021_05,Water,Total,22293.08,ZRW,408762


In [86]:
inventory_shortfalls

,Inventory_Site,Portfolio,Material,Period,Inventory_Units_Required
49,5501,Aseptic,214633,2020_06,342.0
50,5501,Aseptic,214633,2020_07,979.0
51,5501,Aseptic,214633,2020_08,917.0
53,5501,Aseptic,214633,2020_10,177.0
61,5501,Aseptic,214634,2020_06,925.0
...,...,...,...,...,...
26332,5558,Oasis,237990,2020_07,81.0
26340,5558,Oasis,237991,2020_06,523.0
26341,5558,Oasis,237991,2020_07,382.0
26349,5558,Oasis,414850,2020_06,40.0


In [67]:
x = set(shift_lever.Material)
y = set(demand_shortfalls.Material)
z = set(line_loading.Material)
x.intersection(y).intersection(z)

{'090865', '100691', '106513', '106516', '245692'}

In [87]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever['Site_SAP'] = shift_lever['Site_SAP'].fillna(0).astype(int)
try:
    shift_lever = shift_lever.drop(columns=['Portfolio','Demand_Site','Additional_Units_Required'])
except:    
    pass

line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading')).round(2)
line_loading = line_loading[['Line_Portfolio', 'Material', 'Site_SAP', 'Line', 'Year_Period']].drop_duplicates()
line_loading.columns = ['Portfolio', 'Material', 'Site_SAP', 'Line', 'Period']

#demand_shortfalls.Material = demand_shortfalls.Material.astype(str)
inventory_shortfalls.Material = inventory_shortfalls.Material.astype(str)

#shift_lever = pd.merge(shift_lever, line_loading, on=['Site_SAP', 'Line', 'Period'], how='left')

shift_lever = pd.merge(shift_lever, demand_shortfalls, left_on=['Site_SAP', 'Period'], 
                       right_on=['Demand_Site', 'Period'], how='left')

group_cols = shift_lever.columns.tolist()


shift_lever = pd.merge(shift_lever, inventory_shortfalls, on=['Portfolio', 'Period', 'Material'], how='left')
shift_lever = shift_lever.fillna(0)

In [4]:
line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading')).round(2)

NameError: name 'INPUT_PICKLE_DIR' is not defined

In [9]:
sP_portSummarisedIHLineLoading.KeyField.value_counts(dropna=False)

Planned                        28118
EstimatedHoursSpareCapacity     1022
EstimatedUnconsumedCapacity     1022
SpareHours                       511
Capacity                         511
Consumed                         511
Name: KeyField, dtype: int64

In [88]:
shift_lever

,Site_SAP,Line,Period,Weeks_WeeksInPeriod,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,...,Hours_SchedPDT,SafetyHours,Hours_TotalAvailable,Additional_Hours,Demand_Site,Portfolio,Material,Additional_Units_Required,Inventory_Site,Inventory_Units_Required
0,5501,LN03,2020_06,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,202.50,0.97,589.2750,7.0000,5501.0,Cans - Board,090865,0.60,0,0.0
1,5501,LN03,2020_06,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,202.50,0.97,589.2750,7.0000,5501.0,Cans - Shrink,090865,0.67,0,0.0
2,5501,LN03,2020_06,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,202.50,0.97,589.2750,7.0000,5501.0,Cans - Singles,090865,1.00,0,0.0
3,5501,LN03,2020_06,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,202.50,0.97,589.2750,7.0000,5501.0,Glass,090865,4.00,0,0.0
4,5501,LN03,2020_07,4.0,8.0,3.0,7.0,0.0,672.0,0.0,...,159.50,0.97,464.1450,0.0000,5501.0,Cans - Shrink,090865,2.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,5506,LN02,2021_01,4.0,24.0,1.0,7.0,0.0,672.0,0.0,...,159.50,0.97,464.1450,0.0000,5506.0,Cans - Singles,201051,0.67,0,0.0
454,5506,LN02,2021_02,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,119.25,0.97,347.0175,0.0000,0.0,0,0,0.00,0,0.0
455,5506,LN02,2021_03,5.0,24.0,1.0,7.0,0.0,840.0,0.0,...,197.25,0.97,573.9975,0.0000,5506.0,Cans - Singles,201051,0.33,0,0.0
456,5506,LN02,2021_04,4.0,24.0,1.0,7.0,0.0,672.0,0.0,...,159.50,0.97,464.1450,0.0000,0.0,0,0,0.00,0,0.0


In [81]:
inventory_shortfalls.head()

,Inventory_Site,Portfolio,Material,Period,Inventory_Units_Required
49,5501,Aseptic,214633,2020_06,342.0
50,5501,Aseptic,214633,2020_07,979.0
51,5501,Aseptic,214633,2020_08,917.0
53,5501,Aseptic,214633,2020_10,177.0
61,5501,Aseptic,214634,2020_06,925.0


In [3]:
INPUT_PICKLE_DIR = '../input/pickle'
line_loading = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'sP_portSummarisedIHLineLoading')).round(2)

In [5]:
line_loading.Site_SAP.unique()

array([5501, 5502, 5503, 5506, 5546], dtype=int64)

In [352]:
import pandas as pd
import os

INPUT_PICKLE_DIR = '../input/pickle'
DEMAND_SHORTFALL_TRESHOLD = 0

In [409]:
def get_demand_alerts():
    dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)

    # Demand Shortfalls
    demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio', 'Material', 'Uom'], \
                                        columns='KeyField')['Value'].reset_index()

    demand_plan['demand_short_fall'] = -1*(demand_plan['Available'] - demand_plan['Total'])
    demand_plan['demand_short_fall_pct'] = round((demand_plan['Available'] - demand_plan['Total'])*-100/ demand_plan['Available'])

    demand_shortfalls = demand_plan.loc[demand_plan.demand_short_fall_pct > DEMAND_SHORTFALL_TRESHOLD] 

    demand_shortfalls = demand_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'demand_short_fall']]

    demand_shortfalls.Material = demand_shortfalls.Material.astype(str)
    demand_shortfalls.Site_SAP = demand_shortfalls.Site_SAP.astype(str)
    demand_shortfalls.columns = ['Demand_Site', 'Portfolio', 'Demand_Material', 'Year_Period', 'Additional_Units_Required']

    return demand_shortfalls

In [410]:
import numpy as np
INVENTORY_SHORTFALL_THRESHOLD = 5

def get_inventory_alerts():
    # Inventory Shortfalls
    pivot_cols = ['KeyField']
    pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
    value_col = ['Value']
    agg_func = np.sum
    filter_by = {'Uom': 'ZUC'}

    iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)

    df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_ClosingStockTarget', 'ZUC_StockVsTarget'])]

    inventory_shortfalls = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

    inventory_shortfalls = pd.DataFrame(inventory_shortfalls.to_records())

    inventory_shortfalls.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_shortfalls.columns]

    inventory_shortfalls['inventory_shortfall_pct'] = round(inventory_shortfalls['ZUC_StockVsTarget']*-100 / inventory_shortfalls['ZUC_ClosingStockTarget'], 2)

    inventory_shortfalls['inventory_shortfall'] = -1*inventory_shortfalls['ZUC_StockVsTarget']

    inventory_shortfalls = inventory_shortfalls.loc[inventory_shortfalls.inventory_shortfall_pct > INVENTORY_SHORTFALL_THRESHOLD]

    inventory_shortfalls = inventory_shortfalls[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period', 'inventory_shortfall']]

    inventory_shortfalls.Material = inventory_shortfalls.Material.astype(str)
    inventory_shortfalls.Site_SAP = inventory_shortfalls.Site_SAP.astype(str)
    inventory_shortfalls.columns = ['Inventory_Site', 'Portfolio', 'Inventory_Material', 'Year_Period', 'Inventory_Units_Required']

    return inventory_shortfalls

In [493]:
da = get_demand_alerts().head()
ia = get_inventory_alerts().head()

In [494]:
da

,Demand_Site,Portfolio,Demand_Material,Year_Period,Additional_Units_Required
12,5502,REPACK,106513,2020_06,15197.0
603,5506,REPACK,201051,2020_06,12810.0


In [413]:
ia

,Inventory_Site,Portfolio,Inventory_Material,Year_Period,Inventory_Units_Required
49,5501,Aseptic,214633,2020_06,342.0
50,5501,Aseptic,214633,2020_07,979.0
51,5501,Aseptic,214633,2020_08,917.0
53,5501,Aseptic,214633,2020_10,177.0
61,5501,Aseptic,214634,2020_06,925.0


In [414]:
ia['Inventory_Site'] = ['5502', '5502', '5502', '5502', '5503']
ia['Inventory_Material'] = ['245692','245692','245692','245692','245692']
da['Demand_Material'] = ['245692','245692','245692']

In [415]:
da

,Demand_Site,Portfolio,Demand_Material,Year_Period,Additional_Units_Required
12,5502,REPACK,245692,2020_06,15197.0
211,5503,REPACK,245692,2020_06,1787.0
603,5506,REPACK,245692,2020_06,12810.0


In [416]:
ia

,Inventory_Site,Portfolio,Inventory_Material,Year_Period,Inventory_Units_Required
49,5502,Aseptic,245692,2020_06,342.0
50,5502,Aseptic,245692,2020_07,979.0
51,5502,Aseptic,245692,2020_08,917.0
53,5502,Aseptic,245692,2020_10,177.0
61,5503,Aseptic,245692,2020_06,925.0


In [483]:
INPUT_PICKLE_DIR
SHIFT_LEVER_COLUMNS = ['Site_SAP','Line','Year_Period','Weeks_In_Period','Shift_Length','Shifts_In_Day',
                  'Shift_Days_In_Week','Hours_Overtime', 'Hours_Crew','Hours_UnCrewed', 'Loss_Factor','Hours_Down_Time',
                  'Hours_Sched_PDT','Safety_Hours','Hours_Total_Available', 'Hours_Total_Available_Calc','Additional_Hours']

In [484]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever = shift_lever[SHIFT_LEVER_COLUMNS]
shift_lever = shift_lever.drop_duplicates(subset=['Site_SAP','Line','Year_Period'])
shift_lever['Site_SAP'] = shift_lever['Site_SAP'].fillna(0).astype(int).astype(str)

production_capability = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'production_capacity'))
production_capability = production_capability[['Plant', 'Line', 'Material']].drop_duplicates()
production_capability.columns = ['Site_SAP', 'Line', 'Material']
production_capability.Site_SAP = production_capability.Site_SAP.astype(str)

shift_lever = pd.merge(shift_lever, production_capability, on=['Site_SAP', 'Line'])

In [485]:
shift_lever = pd.merge(shift_lever, da.drop('Portfolio', axis=1), 
                       left_on=['Material','Year_Period'], right_on=['Demand_Material','Year_Period'], 
                       how='left')

In [486]:
shift_lever = pd.merge(shift_lever, ia.drop('Portfolio', axis=1), 
                       left_on=['Material','Year_Period'], right_on=['Inventory_Material','Year_Period'], 
                       how='left')

In [487]:
shift_lever = shift_lever.dropna()
short_cols = ['Demand_Site', 'Demand_Material', 'Additional_Units_Required', 
              'Inventory_Site', 'Inventory_Material', 'Inventory_Units_Required']

for col in short_cols:
        shift_lever[col] = shift_lever[col].astype(str)


In [488]:
shift_lever['Demand_Shortfall'] = shift_lever[['Demand_Material', 'Demand_Site', 'Additional_Units_Required']].agg('_'.join, axis=1)
shift_lever['Inventory_Shortfall'] = shift_lever[['Inventory_Material', 'Inventory_Site', 'Inventory_Units_Required']].agg('_'.join, axis=1)
shift_lever = shift_lever.drop(short_cols, axis=1)

In [490]:
shift_lever.head()

,Site_SAP,Line,Year_Period,Weeks_In_Period,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,Loss_Factor,Hours_Down_Time,Hours_Sched_PDT,Safety_Hours,Hours_Total_Available,Hours_Total_Available_Calc,Additional_Hours,Material,Demand_Shortfall,Inventory_Shortfall
16345,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5502-15197.0,245692-5502-342.0
16346,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5502-15197.0,245692-5503-925.0
16347,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5503-1787.0,245692-5502-342.0
16348,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5503-1787.0,245692-5503-925.0
16349,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0,245692-5502-342.0


In [491]:
agg_cols = ['Material', 'Demand_Shortfall', 'Inventory_Shortfall']

for col in agg_cols:
    shift_lever[col] = shift_lever[col].astype(str)
    shift_lever[col] = shift_lever.groupby(SHIFT_LEVER_COLUMNS)[col].transform(lambda x : ' '.join(set(x))) 

In [492]:
shift_lever

,Site_SAP,Line,Year_Period,Weeks_In_Period,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,Loss_Factor,Hours_Down_Time,Hours_Sched_PDT,Safety_Hours,Hours_Total_Available,Hours_Total_Available_Calc,Additional_Hours,Material,Demand_Shortfall,Inventory_Shortfall
16345,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
16346,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
16347,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
16348,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
16349,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
16350,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,0.25,363,119.25,0.97,347.0175,347.0175,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
17673,5501,LN09,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,573.9975,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
17674,5501,LN09,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,573.9975,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
17675,5501,LN09,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,573.9975,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0
17676,5501,LN09,2020_06,5,24,1,7,0,840.0,0.0,0.25,51,197.25,0.97,573.9975,573.9975,0,245692,245692-5506-12810.0 245692-5502-15197.0 245692...,245692-5503-925.0 245692-5502-342.0


In [456]:
shift_lever['Demand_Material'] = shift_lever['Demand_Material'].astype(str)
shift_lever['Inventory_Material'] = shift_lever['Inventory_Material'].astype(str)

shift_lever['Material'] = shift_lever.groupby(group_cols)['Material'].transform(lambda x : ' '.join(set(x))) 
shift_lever['Demand_Material'] = shift_lever.groupby(group_cols)['Demand_Material'].transform(lambda x : ' '.join(set(x))) 
shift_lever['Inventory_Material'] = shift_lever.groupby(group_cols)['Demand_Material'].transform(lambda x : ' '.join(set(x))) 

In [457]:
shift_lever['Demand_Site'] = shift_lever['Demand_Site'].astype(str)


shift_lever['Demand_Site'] = shift_lever.groupby(group_cols)['Demand_Site'].transform(lambda x : ' '.join(set(x)))
shift_lever['Additional_Units_Required'] = shift_lever['Additional_Units_Required'].astype(str)
shift_lever['Additional_Units_Required'] = shift_lever.groupby(group_cols)['Additional_Units_Required'].transform(lambda x : ' '.join(set(x))) 


In [458]:

shift_lever['Inventory_Site'] = shift_lever['Inventory_Site'].astype(str)

shift_lever['Inventory_Site'] = shift_lever.groupby(group_cols)['Inventory_Site'].transform(lambda x : ' '.join(set(x)))
shift_lever['Inventory_Units_Required'] = shift_lever['Inventory_Units_Required'].astype(str)
shift_lever['Inventory_Units_Required'] = shift_lever.groupby(group_cols)['Inventory_Units_Required'].transform(lambda x : ' '.join(set(x))) 


In [459]:

#shift_lever['Portfolio'] = shift_lever.groupby(group_cols)['Portfolio'].transform(lambda x : ' '.join(set(x))) 


In [460]:
shift_lever = shift_lever.drop_duplicates()

In [461]:
shift_lever

,Site_SAP,Line,Year_Period,Weeks_In_Period,Shift_Length,Shifts_In_Day,Shift_Days_In_Week,Hours_Overtime,Hours_Crew,Hours_UnCrewed,...,Hours_Total_Available,Hours_Total_Available_Calc,Additional_Hours,Material,Demand_Site,Demand_Material,Additional_Units_Required,Inventory_Site,Inventory_Material,Inventory_Units_Required
41,5501,LN08,2020_06,5,24,1,7,0,840.0,0.0,...,347.0175,347.0175,0,245692,5503 5502 5506,245692,12810.0 15197.0 1787.0,5503 5502,245692,342.0 925.0
129,5501,LN09,2020_06,5,24,1,7,0,840.0,0.0,...,573.9975,573.9975,0,245692,5503 5502 5506,245692,12810.0 15197.0 1787.0,5503 5502,245692,342.0 925.0
175,5501,LN10,2020_06,5,24,1,7,0,840.0,0.0,...,347.0175,347.0175,0,245692,5503 5502 5506,245692,12810.0 15197.0 1787.0,5503 5502,245692,342.0 925.0
230,5503,LN04,2020_06,5,24,1,7,0,840.0,0.0,...,347.0175,347.0175,0,245692,5503 5502 5506,245692,12810.0 15197.0 1787.0,5503 5502,245692,342.0 925.0
275,5503,LN05,2020_06,5,24,1,7,0,840.0,0.0,...,573.9975,573.9975,0,245692,5503 5502 5506,245692,12810.0 15197.0 1787.0,5503 5502,245692,342.0 925.0


['5506', '5567', '5569', '5503', '5502']

26277

In [196]:
import pyodbc
def get_conversion_ratio(material, from_uom, to_uom, schema_name='material'):
    conn = pyodbc.connect('Driver={SQL Server};'
                        f'Server={SERVER_NAME};'
                        f'Database={DATABASE_NAME};'
                        'Trusted_Connection=yes;')

    sql = f'''SELECT [CombinedConversion]
                FROM [{DATABASE_NAME}].[{schema_name}].[{CONVERSION_TABLE}]
                WHERE [Material] = '{material}'
                AND [From_Uom] = '{from_uom}'
                AND [To_Uom] = '{to_uom}'
                '''

    data = pd.read_sql(sql,conn)

    conv_ration = round(data.CombinedConversion.values[0],4)

    conn.close()

    return conv_ration

In [197]:
SERVER_NAME = 'CVAPWPEQR001'
DATABASE_NAME = 'PACSQL_MTP'
CONVERSION_TABLE = 'Output_ecc_ConversionTable'

get_conversion_ratio('265779', 'ZUC', 'ZRW')

0.4732

In [230]:
INPUT_LEVER_DIR = '../input/levers'
import_lever_metadata = pd.read_csv(os.path.join(INPUT_LEVER_DIR, 'import_cofill_lever.csv'))

In [231]:
import_lever_metadata = import_lever_metadata.fillna('')
import_lever_metadata['Site_SAP_From'] = import_lever_metadata['Site_SAP_From'].astype(str).str.strip()

import_lever_metadata = import_lever_metadata.loc[import_lever_metadata.Import_Cofill.isin(['Import', 'Cofill'])]
import_lever_metadata['Site'] = import_lever_metadata[['Site_SAP_From', 'Site_Name']].agg(' - '.join, axis=1)

In [234]:
import_lever_metadata['Site'].values[0]

'15N-DONGEN'

In [226]:
import_lever_metadata['Site_SAP_From'] = import_lever_metadata['Site_SAP_From'].astype(str).str.strip()



In [206]:
x.FromCountry.unique()

array(['NL', 'BE', 'FR', 'GB', 'IE', nan, 'GE', 'IN'], dtype=object)

In [209]:
"asdad|asdasd".split('|')[0]

'asdad'

In [267]:
MOVEMENT_TABLE = 'temp_mP_Summarised_materiallevel'
def get_site_region_mapping():
    try:      

        conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')
        
        sql = f'''
                SELECT DISTINCT
                        TRIM([ToCountry])   as country   
                        ,TRIM([To_GeoRegion]) as region   
                        ,TRIM([Site_SAP_To]) as site
                FROM [{DATABASE_NAME}].[dbo].[{MOVEMENT_TABLE}]
                UNION
                SELECT DISTINCT
                        TRIM([FromCountry])  as country   
                        ,TRIM([From_GeoRegion]) as region
                        ,TRIM([Site_SAP_From]) as site
                FROM [{DATABASE_NAME}].[dbo].[{MOVEMENT_TABLE}]
                '''
        
        data = pd.read_sql(sql,conn)           
        
        conn.close()
    except Exception as e:        
        raise Exception(f"Database error: Unable to get Site Region mapping from [{DATABASE_NAME}].{MOVEMENT_TABLE} Exception {e}")

    return data

In [280]:
x = get_site_region_mapping()
x

,country,region,site
0,ER,None,EXPO_EX
1,ES,None,I54
2,BE,None,ANTW_RP
3,BE,None,52B
4,GB,None,P0_5556
...,...,...,...
57,None,None,DIS_NL
58,GB,South,5560
59,FR,None,44F
60,GE,None,KART_GE


In [282]:
x.loc[x.site.isin([
    '15N',
'19B',
'44F',
'AILI_IE',
'APPL_GB',
'CACO_FR',
'COTT_GB',
'DAYL_GB',
'DIS_NL',
'EDRI_GB',
'KART_GE',
'MBEV_IN',
'MILK_GB',
'NORB_GB',
'PRIN_GB',
'WILD_GE',
'WINT_NL']
)]
    

,country,region,site
30,GB,None,PRIN_GB
31,None,None,WILD_GE
34,None,None,WINT_NL
35,FR,None,CACO_FR
36,IN,None,MBEV_IN
37,GB,None,NORB_GB
40,BE,None,19B
43,GB,None,COTT_GB
44,IE,None,AILI_IE
46,GB,None,MILK_GB


In [262]:
import_lever = pd.read_csv(os.path.join(INPUT_LEVER_DIR, 'import_cofill_lever.csv'))

In [263]:
import_lever

,FromCountry,ToCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom


In [264]:
from_country = 'qwe'
from_region = 'ert'
to_region = 'dsf'
from_site = 'asd'
to_site = 'asd'
year_period = 'yp'
material = 'ert'

units = '900'

In [265]:
new_row = {'FromCountry':from_country, 'ToCountry':'GB', 'SiteCountry':'', 'From_GeoRegion':from_region, 'To_GeoRegion':to_region,
                    'Site_SAP_From': from_site, 'Site_SAP_To': to_site, 'Year_Period': year_period, 'Summary_Portfolio':'', 
                    'Material': material,'KeyField':'ZUC', 'Value':units, 'Uom':'ZUC', 'ToRepack_Flag':0
                    }    
        
print(new_row)

print(import_lever.columns)

import_lever = import_lever.append(new_row, ignore_index=True) 

{'FromCountry': 'qwe', 'ToCountry': 'GB', 'SiteCountry': '', 'From_GeoRegion': 'ert', 'To_GeoRegion': 'dsf', 'Site_SAP_From': 'asd', 'Site_SAP_To': 'asd', 'Year_Period': 'yp', 'Summary_Portfolio': '', 'Material': 'ert', 'KeyField': 'ZUC', 'Value': '900', 'Uom': 'ZUC', 'ToRepack_Flag': 0}
Index(['FromCountry', 'ToCountry', 'From_GeoRegion', 'To_GeoRegion',
       'Site_SAP_From', 'Site_SAP_To', 'Year_Period', 'Summary_Portfolio',
       'Material', 'KeyField', 'Value', 'Uom'],
      dtype='object')


In [286]:
to_sites = [None, '5556', '5558', '5559', '5560', '5569', '5506', '5546', '5501', '5502', '5503']
[{'label': i, 'value': i} for i in to_sites if i]

[{'label': '5556', 'value': '5556'},
 {'label': '5558', 'value': '5558'},
 {'label': '5559', 'value': '5559'},
 {'label': '5560', 'value': '5560'},
 {'label': '5569', 'value': '5569'},
 {'label': '5506', 'value': '5506'},
 {'label': '5546', 'value': '5546'},
 {'label': '5501', 'value': '5501'},
 {'label': '5502', 'value': '5502'},
 {'label': '5503', 'value': '5503'}]

In [5]:
sP_portSummarisedIHLineLoading

,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,5501,LN03,2020_06,Planned,2.391840e+05,ZRW
1,Aseptic,5501,LN03,2020_06,Planned,2.906251e+06,ZUC
2,Aseptic,5501,LN03,2020_07,Planned,2.929728e+05,ZRW
3,Aseptic,5501,LN03,2020_07,Planned,3.009829e+06,ZUC
4,Aseptic,5501,LN03,2020_08,Planned,3.669637e+05,ZRW
...,...,...,...,...,...,...,...
3303,Water,5546,LN01,2021_02,SpareHours,0.000000e+00,Hours
3304,Water,5546,LN01,2021_03,SpareHours,0.000000e+00,Hours
3305,Water,5546,LN01,2021_04,SpareHours,0.000000e+00,Hours
3306,Water,5546,LN01,2021_05,SpareHours,0.000000e+00,Hours


In [6]:
sP_portSummarisedIHLineLoading.KeyField.unique()

array(['Planned', 'Capacity', 'Consumed', 'EstimatedHoursSpareCapacity',
       'EstimatedUnconsumedCapacity', 'SpareHours'], dtype=object)

In [11]:
def get_production_line_plan(level='Line_Portfolio', uom='Hours'):
    pivot_cols = ['KeyField']
    pivot_rows = [level, 'Site_SAP', 'Line', 'Year_Period']
    value_col = ['Value']
    agg_func = np.sum
    filter_by = {'Uom': uom}

    df = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE))
    
    df = summarise_df_for_visualisation(df)

    wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                        columns=pivot_cols, aggfunc=agg_func)

    production_line_plan = pd.DataFrame(wide_df.to_records())

    production_line_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                        for hdr in production_line_plan.columns]

    production_line_plan['Consumed'] = production_line_plan['Consumed'].astype(float)
    production_line_plan['Capacity'] = production_line_plan['Capacity'].astype(float)

    #Calcuated fields
    production_line_plan['Capacity_Utilisation'] = production_line_plan['Consumed']*100 / production_line_plan['Capacity']

    production_line_plan['Available_Vol'] = production_line_plan['EstimatedUnconsumedCapacity']
    production_line_plan['Available_Vol_with_Safety_Hours'] = production_line_plan['EstimatedUnconsumedCapacity'] + production_line_plan['EstimatedHoursSpareCapacity']


    production_line_plan = production_line_plan.fillna(0)

    return production_line_plan

In [20]:

pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

df = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE))

In [29]:
df.head()

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,211313,5501,LN03,2020_12,Planned,11963.794104,ZRW
1,Aseptic,211313,5501,LN03,2020_12,Planned,12642.000000,ZUC
2,Aseptic,211313,5501,LN03,2021_01,Planned,17862.412095,ZRW
3,Aseptic,211313,5501,LN03,2021_01,Planned,18875.000000,ZUC
4,Aseptic,211313,5501,LN03,2021_02,Planned,23326.654078,ZRW


In [22]:
df.KeyField.value_counts(dropna=False)

Planned                        28118
EstimatedHoursSpareCapacity     1022
EstimatedUnconsumedCapacity     1022
SpareHours                       511
Capacity                         511
Consumed                         511
Name: KeyField, dtype: int64

In [28]:
df.loc[df.Line_Portfolio == 'Aseptic'].KeyField.value_counts(dropna=False)

Planned                        138
EstimatedUnconsumedCapacity     34
EstimatedHoursSpareCapacity     34
SpareHours                      17
Consumed                        17
Capacity                        17
Name: KeyField, dtype: int64

In [38]:
# Function to fill NA values in dataframe
def fill_na_by_type(df, str_na='', float_na=0, int_na=0, bool_na=False):
    float_cols = df.select_dtypes(include=['float64']).columns
    str_cols = df.select_dtypes(include=['object']).columns

    df.loc[:, float_cols] = df.loc[:, float_cols].fillna(float_na)
    df.loc[:, str_cols] = df.loc[:, str_cols].fillna(str_na)

    return df

# Function to trim all string columns in a dataframe
def trim_strings_df(df):
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    return df


In [39]:
import pyodbc

SERVER_NAME = 'CVAPWPEQR001'
DATABASE_NAME = 'PACSQL_MTP'

# Function to get data from the database for a given table
def get_datatable_from_db(table_name, where_string=''):
    
    try:
        conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')

        sql = f''' SELECT * FROM [{DATABASE_NAME}].{table_name} {where_string} '''

        data = pd.read_sql(sql,conn)

        data = trim_strings_df(data)
        data = fill_na_by_type(data)

        conn.close()
    except Exception as e:
        raise Exception(f"Database Error: Unable to fetch data for [{DATABASE_NAME}].{table_name} Exception {e}")

    return data

In [40]:
ll = get_datatable_from_db('dbo.temp_sP_portSummarisedIHLineLoading_materiallevel_REFRESH')

In [41]:
ll.head()

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,211313,5501,LN03,2020_12,Planned,11963.7941036224,ZRW
1,Aseptic,211313,5501,LN03,2020_12,Planned,12642,ZUC
2,Aseptic,211313,5501,LN03,2021_01,Planned,17862.4120950699,ZRW
3,Aseptic,211313,5501,LN03,2021_01,Planned,18875,ZUC
4,Aseptic,211313,5501,LN03,2021_02,Planned,23326.6540784836,ZRW


In [42]:
ll.KeyField.value_counts(dropna=False)

Planned                        28118
EstimatedHoursSpareCapacity     1022
EstimatedUnconsumedCapacity     1022
SpareHours                       511
Capacity                         511
Consumed                         511
Name: KeyField, dtype: int64

In [12]:
def get_shift_plan(input_pickle_dir, pickle_file_name='MTP_WF003_05_Visualise Baseline Plan_ShiftData'):
    pivot_cols = ['Hours_Type']
    pivot_rows = ['Site_SAP', 'Line', 'Year_Period']
    value_col = ['Value']
    agg_func = np.sum
    filter_by = {}

    df = pd.read_pickle(os.path.join(input_pickle_dir, pickle_file_name))

    # Conditional categories
    conditions = [
        df.KeyField == 'Hours_Spare',
        df.KeyField == 'Hours_DownTime', 
        df.KeyField == 'Hours_SchedPDT',
        df.KeyField == 'Hours_UnCrewed',
        ]
    choices = ['Spare_Hours','Planned_Downtime', 'Loss_Factor', 'Uncrewed']

    df['Hours_Type'] = np.select(conditions, choices, default='Hours_Other')

    #'Filter'
    df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

    # pivot
    wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                        columns=pivot_cols, aggfunc=agg_func)


    shift_plan = pd.DataFrame(wide_df.to_records())

    shift_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                        for hdr in shift_plan.columns]

    shift_plan = shift_plan.fillna(0)

    return shift_plan

In [13]:
INPUT_PICKLE_DIR = '../input/pickle'
SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE = 'sP_portSummarisedIHLineLoading'
MOVEMENT_PICKLE = 'mP_Summarised'
SHIFT_PLAN_PICKLE = 'MTP_WF003_05_Visualise Baseline Plan_ShiftData'
SHIFT_LEVER = 'shift_lever.csv'

In [35]:
df = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE))

In [17]:
production_line_plan = get_production_line_plan()
production_line_plan.head()

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,EstimatedHoursSpareCapacity,EstimatedUnconsumedCapacity,Planned,SpareHours,Capacity_Utilisation,Available_Vol,Available_Vol_with_Safety_Hours
0,Aseptic,5501,LN03,2020_12,332.0,20.0,46257.0,799831.0,52146.0,18.0,6.024096,799831.0,846088.0
1,Aseptic,5501,LN03,2021_01,334.0,31.0,62748.0,894121.0,91079.0,21.0,9.281437,894121.0,956869.0
2,Aseptic,5501,LN03,2021_02,363.0,34.0,68881.0,979688.0,101826.0,23.0,9.366391,979688.0,1048569.0
3,Aseptic,5501,LN03,2021_03,432.0,66.0,83258.0,1105589.0,199878.0,28.0,15.277778,1105589.0,1188847.0
4,Aseptic,5501,LN03,2021_04,348.0,60.0,68849.0,895828.0,185186.0,22.0,17.241379,895828.0,964677.0


In [50]:
production_line_plan.Year_Period.unique()

array(['2020_12', '2021_01', '2021_02', '2021_03', '2021_04', '2021_05',
       '2021_06', '2021_07', '2021_08', '2021_09', '2021_10', '2021_11',
       '2021_12', '2022_01', '2022_02', '2022_03', '2022_04'],
      dtype=object)

In [57]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever

,Site_SAP,Line,Year_Period,Weeks_In_Period,Shift_Days,Shift_Length,Hours_Overtime,Hours_Crew,Loss_Factor,Hours_Down_Time,Hours_Sched_PDT,Safety_Hours,Hours_Total_Available,Additional_Hours_Used,Hours_Total_Available_Calc,Hours_UnCrewed,Additional_Hours
0,5501,LN01,2021_01,4,28,24.0,0,672.0,0.28,462.0,58.80,1.0,151.20,0,0,0,0
1,5501,LN01,2021_02,4,28,24.0,0,672.0,0.28,48.0,174.72,1.0,449.28,0,0,0,0
2,5501,LN01,2021_03,5,35,24.0,0,840.0,0.28,115.0,203.00,1.0,522.00,0,0,0,0
3,5501,LN01,2021_04,4,28,24.0,0,672.0,0.28,86.0,164.08,1.0,421.92,0,0,0,0
4,5501,LN01,2021_05,4,28,24.0,0,672.0,0.28,42.0,176.40,1.0,453.60,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,5546,LN01,2021_12,5,20,21.0,0,420.0,0.24,107.0,75.12,1.0,237.88,0,0,0,0
492,5546,LN01,2022_01,4,16,21.0,0,336.0,0.24,17.0,76.56,1.0,242.44,0,0,0,0
493,5546,LN01,2022_02,4,16,21.0,0,336.0,0.24,19.0,76.08,1.0,240.92,0,0,0,0
494,5546,LN01,2022_03,5,20,21.0,0,420.0,0.24,93.0,78.48,1.0,248.52,0,0,0,0


In [60]:
production_line_plan.Site_SAP = production_line_plan.Site_SAP.astype(str)
shift_lever.Site_SAP = shift_lever.Site_SAP.astype(str)

In [61]:
x = pd.merge(production_line_plan, shift_lever, on=['Site_SAP', 'Line', 'Year_Period'], how='inner')
x.head()

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,EstimatedHoursSpareCapacity,EstimatedUnconsumedCapacity,Planned,SpareHours,...,Hours_Crew,Loss_Factor,Hours_Down_Time,Hours_Sched_PDT,Safety_Hours,Hours_Total_Available,Additional_Hours_Used,Hours_Total_Available_Calc,Hours_UnCrewed,Additional_Hours
0,Aseptic,5501,LN03,2021_01,334.0,31.0,62748.0,894121.0,91079.0,21.0,...,672.0,0.41,100.0,234.52,1.0,337.48,0,0,0,0
1,Aseptic,5501,LN03,2021_02,363.0,34.0,68881.0,979688.0,101826.0,23.0,...,672.0,0.41,48.0,255.84,1.0,368.16,0,0,0,0
2,Aseptic,5501,LN03,2021_03,432.0,66.0,83258.0,1105589.0,199878.0,28.0,...,840.0,0.41,121.0,294.79,1.0,424.21,0,0,0,0
3,Aseptic,5501,LN03,2021_04,348.0,60.0,68849.0,895828.0,185186.0,22.0,...,672.0,0.41,140.0,218.12,1.0,313.88,0,0,0,0
4,Aseptic,5501,LN03,2021_05,284.0,31.0,55306.0,773706.0,94664.0,18.0,...,672.0,0.41,203.0,192.29,1.0,276.71,0,0,0,0


In [72]:
x= x[['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period', 
  'Capacity', 'Consumed', 'SpareHours', 'Hours_Down_Time', 'Hours_Sched_PDT', 
   'Hours_Crew', 'Weeks_In_Period'
 ]]

x.columns = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period', 
            'Capacity', 'Consumed', 'SpareHours', 'Planned_Downtime', 'LossFactor',
             'Hours_Crew', 'Weeks_In_Period'
            ]

x

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,SpareHours,Planned_Downtime,LossFactor,Hours_Crew,Weeks_In_Period
0,Aseptic,5501,LN03,2021_01,334.0,31.0,21.0,100.0,234.52,672.0,4
1,Aseptic,5501,LN03,2021_02,363.0,34.0,23.0,48.0,255.84,672.0,4
2,Aseptic,5501,LN03,2021_03,432.0,66.0,28.0,121.0,294.79,840.0,5
3,Aseptic,5501,LN03,2021_04,348.0,60.0,22.0,140.0,218.12,672.0,4
4,Aseptic,5501,LN03,2021_05,284.0,31.0,18.0,203.0,192.29,672.0,4
...,...,...,...,...,...,...,...,...,...,...,...
955,SE,5546,LN01,2022_03,0.0,0.0,0.0,93.0,78.48,420.0,5
956,Water,5546,LN01,2022_03,243.0,243.0,0.0,93.0,78.48,420.0,5
957,REPACK,5546,LN01,2022_04,0.0,0.0,0.0,17.0,76.56,336.0,4
958,SE,5546,LN01,2022_04,0.0,0.0,0.0,17.0,76.56,336.0,4


In [74]:
x

,Line_Portfolio,Site_SAP,Line,Year_Period,Capacity,Consumed,SpareHours,Planned_Downtime,LossFactor,Hours_Crew,Weeks_In_Period,Uncrewed
0,Aseptic,5501,LN03,2021_01,334.0,31.0,21.0,100.0,234.52,672.0,4,0.0
1,Aseptic,5501,LN03,2021_02,363.0,34.0,23.0,48.0,255.84,672.0,4,0.0
2,Aseptic,5501,LN03,2021_03,432.0,66.0,28.0,121.0,294.79,840.0,5,0.0
3,Aseptic,5501,LN03,2021_04,348.0,60.0,22.0,140.0,218.12,672.0,4,0.0
4,Aseptic,5501,LN03,2021_05,284.0,31.0,18.0,203.0,192.29,672.0,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
955,SE,5546,LN01,2022_03,0.0,0.0,0.0,93.0,78.48,420.0,5,420.0
956,Water,5546,LN01,2022_03,243.0,243.0,0.0,93.0,78.48,420.0,5,420.0
957,REPACK,5546,LN01,2022_04,0.0,0.0,0.0,17.0,76.56,336.0,4,336.0
958,SE,5546,LN01,2022_04,0.0,0.0,0.0,17.0,76.56,336.0,4,336.0


In [76]:
168*4/2

336.0

In [110]:
x = '55509.0'

x.isdigit()

False